## DreliaCalc Report

In [ ]:
#import arrow; print(arrow.now('Europe/Vienna').format('DD.MM.YYYY - HH:mm'))
from datetime import datetime; print(datetime.now().strftime('%d.%m.%Y - %H:%M'))

In [ ]:
import dmyplant2
import pandas as pd
import numpy as np
from tabulate import tabulate
from pprint import pprint as pp

In [ ]:
# load input data from files
dval = dmyplant2.Validation.load_def_csv("out.csv")
failures = dmyplant2.Validation.load_failures_csv("failures.csv")
psize = (12,8)

## Validation Engines:

In [ ]:
dmyplant2.cred()
mp = dmyplant2.MyPlant(0)

In [ ]:
vl = dmyplant2.Validation.from_dval(mp,dval, cui_log=False) 

In [ ]:
mp._token;

In [ ]:
d = vl.dashboard
d.T                  

In [ ]:
mp.fetch_available_data();

In [ ]:
vl.quick_report()

## Success Run Plot, beta=1.21 & 1.5 with failures 

In [ ]:
from tabulate import tabulate
print(tabulate(failures, headers=failures.columns))

In [ ]:
dmyplant2.demonstrated_Reliabillity_Plot(vl, 
            beta=1.21, T=30000, s=1000, ft=failures, cl=[10,50,90], factor=1.3);
dmyplant2.demonstrated_Reliabillity_Plot(vl, 
            beta=1.5, T=30000, s=1000, ft=failures, cl=[10,50,90], factor=1.3);

In [ ]:
dd = d[['Name','oph parts','BMEP', 'LOC','Engine Type','Engine ID']].copy()
dd['Name'] = d['Name'] + ' ' + d['Engine Type'] + ' ' + d['Engine ID']
dc =  pd.concat([dd, dval[['Old PU first replaced OPH','Old PUs replaced before upgrade']]],axis=1).set_index('Name').sort_values(by = "oph parts",ascending=True)
dc.plot.barh(y=['oph parts','Old PU first replaced OPH'],figsize=psize, xlim=(0,18000), color=['blue','brown'], grid=True, title="PU RCA Running hours vs old PU replacement hours");

In [ ]:
dc.plot.barh(y=['LOC'],figsize=psize, xlim=(0,0.5), color=['red'], grid=True, title='Lube Oil Consumption, Myplant AVG Calculated Values');

## Detailed plot

In [ ]:
# Remove the semicolon on the next line to see all serial numbers
d[['Name','Engine Type','Engine ID','serialNumber']].sort_values(by='Name');

In [ ]:

#e=dmyplant2.Engine.from_sn(mp,1184199)
e=vl.eng_serialNumber(1184199)
print(f"{e}")
e.dash

In [ ]:
dtrips = e.batch_hist_alarms(p_severities=[800], p_offset=0, p_limit=5)
dtrips['datetime'] = pd.to_datetime(dtrips['timestamp'] * 1000000.0).dt.strftime("%m-%d-%Y %H:%m")
print(tabulate(dtrips[['datetime', 'message', 'name','severity']]))


In [ ]:
dtrips = e.batch_hist_alarms(p_severities=[600], p_offset=0, p_limit=10)
dtrips['datetime'] = pd.to_datetime(dtrips['timestamp'] * 1000000.0).dt.strftime("%m-%d-%Y %H:%m:%S")
print(tabulate(dtrips[['datetime', 'message', 'name','severity']]))

In [ ]:
import arrow
data = ['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif']
dset = [
    {'col':['Power_PowerAct'], 'ylim':(0,5000)},
    {'col':['Hyd_PressCrankCase'],'ylim': [-100, 40]},
    {'col':['Hyd_PressOilDif'], 'ylim':(0,2)},
    {'col':['Oph_parts']}
]

df0 = e.hist_data(
    itemIds= e.get_dataItems(data), 
    p_from=arrow.get(e.val_start).to('Europe/Vienna'), 
    #p_from=arrow.get(e['val start']).to('Europe/Vienna'), 
    p_to=arrow.now('Europe/Vienna'),
    timeCycle=600,
    forceReload=False
)

# Set Type of time column to DateTime
df0['Oph_parts'] = df0.Count_OpHour - e.oph_start


In [ ]:
fig = dmyplant2.dbokeh_chart(
    df0, dset, title = e, figsize=psize)
dmyplant2.bokeh_show(fig)

In [ ]:
dmyplant2.chart( df0, dset, title = e, figsize=psize);

In [ ]:
# Lube Oil Consuption data
dloc = e.Validation_period_LOC()

dmyplant2.chart(dloc, [
    {'col': ['OilConsumption','LOC'],'ylim': (0,0.5)},
    {'col': ['Pow'],'ylim': (0,8000)},
],
title = e,
figsize = psize);

In [ ]:
%%time
# exit for the moment, because code does make sense for now.
import sys
sys.exit(0)

dat = {
    161: ['CountOph','h'], 
    102: ['PowerAct','kW'],
    107: ['Various_Values_SpeedAct','rpm'],
    217: ['Hyd_PressCrankCase','mbar'],
    16546: ['Hyd_PressOilDif','bar']
}

dset2 = [
    {'col': ['PowerAct'],'ylim': [0, 5000]},
    {'col': ['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'darkblue'},
    {'col': ['CountOph'],'ylim': [0, 500]},
    {'col': ['Hyd_PressCrankCase'],'ylim': [-80, 60]},
    {'col': ['Hyd_PressOilDif'],'ylim': [0, 2]}
]

df2 = e.hist_data(
    itemIds=dat, 
    #p_from=arrow.get('2021-03-05 05:28').to('Europe/Vienna'),
    p_from=arrow.get(e.valstart_ts),
    #p_to=arrow.get('2021-03-05 05:30').to('Europe/Vienna'),
    p_to=arrow.now(),
    timeCycle=60)


# Set Type of time column to DateTime
df2['CountOph'] = df2.CountOph - e.oph_start
print(f"Number of Data points: {df2['datetime'].count()}")

In [ ]:
# exit for the moment, because code does make sense for now.
import sys
#sys.exit(0)

dmyplant2.chart(df2, dset2, x = 'datetime', title = e, grid = True, figsize = (18,10));

In [ ]:
%%time
#lindex = df2['time'].values.searchsorted(1618856220030)
%timeit df2['CountOph'].iloc[df2['time'].values.searchsorted(1600000000000)]